# Minimum Dominating Set Challenge

## Setup

In [ ]:
from typing import Set, Tuple, Optional, List

from itertools import product

import numpy as np
from numpy import random as rand
from scipy.sparse import dok_matrix, csr_matrix, bmat

import networkx as nx

from bokeh.io import show, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4

output_notebook()

In [ ]:
rand.seed(42)

In [ ]:
n_nodes = 5
n_edges = 5

n_edge_max = 3

In [ ]:
def generate_graph(n_nodes: int, n_edges: int, n_edge_max: int = 5) -> Set[Tuple[int]]:
    """Creates random edges for graph.
    
    Node labels are from 0 to n_nodes - 1 
    
    Arguments:
        n_nodes: Number of nodes
        n_edges: Number of edges
        n_edge_max: Maximal number of edges for graph
        
    Todo:
        What about connected / unconnected graphs?
        Am I creating the right graph? 
        Think about ill input conditions.
    """
    assert n_edge_max > 0
    assert n_nodes * n_edge_max // 2 > n_edges

    nodes = {n: 0 for n in range(n_nodes)}
    edges = set()

    while len(edges) < n_edges:
        choices = [n for n, count in nodes.items() if count <= n_edge_max]
        e1, e2 = (int(e) for e in rand.choice(choices, size=2, replace=False))
        edge = (e2, e1) if e2 < e1 else (e1, e2)
        if edge not in edges:
            edges.add(edge)
            nodes[e1] += 1
            nodes[e2] += 2

    return edges


test_graph = generate_graph(n_nodes, n_edges, n_edge_max=n_edge_max)

In [ ]:
def get_plot(graph: Set[Tuple[int]], color_nodes: Optional[List[int]] = None) -> Plot:
    """Creates bokeh graph plot from edge set.
    
    Arguments:
        graph: The graph to plot.
        color_nodes: Colors given nodes green, default is white.
    """
    color_nodes = color_nodes or []

    G = nx.Graph(list(graph))
    edge_attrs = {}
    node_attrs = {}

    for start_node, end_node, _ in G.edges(data=True):
        edge_attrs[(start_node, end_node)] = "black"

    for node in G.nodes:
        node_attrs[node] = "green" if node in color_nodes else "white"

    nx.set_edge_attributes(G, edge_attrs, "edge_color")
    nx.set_node_attributes(G, node_attrs, "node_color")

    # Show with Bokeh
    plot = Plot(
        plot_width=400,
        plot_height=400,
        x_range=Range1d(-1.1, 1.1),
        y_range=Range1d(-1.1, 1.1),
    )

    graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))
    graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="node_color")
    plot.renderers.append(graph_renderer)

    node_hover_tool = HoverTool(tooltips=[("index", "@index")])
    plot.add_tools(node_hover_tool)

    return plot


plot = get_plot(test_graph)
show(plot)

Form of the matrix which can be plugged in the annealer.

The condition to minimize is 

$$
    \sum_{i=0}^{N_n} x_i  \quad \Leftrightarrow \quad \vec x \cdot \mathbb{1} \cdot \vec x
$$

with $x_i = 1, 0$ specifying wethere a node is in the dominating set or not.

This condition needs to be minimized under the constraint

$$
    x_i + \sum_{j \in \mathcal{N}_i} x_j \geq 1 \, \forall_{i} \in V
$$
where $\mathcal{N}_i$ is the neighborhood of $i$.
This is equivalent ot saying that for all vertex, the vertex at least one neighboring vertex must be included in the dominating set.


This constraint can be mapped to
$$
    x_i  - s_{i} - 1 + \sum_{j \in \mathcal{N}_i} x_j = 0 \, , \quad s_{i} \geq 0 \, .
$$
So we have to map each $s$ to integers.
The above equation can only be zero if at least on of $x = 1$.

Thus our D-Wave function is

$$
    \text{argmin}_{\vec x, \vec s}\left[
        \sum_{i=0}^{N_n-1} x_i
        + p
        \sum_{i=0}^{N_n-1} \left(x_i  - s_{i} - 1  + \sum_{j \in\mathcal{N}_i} x_j\right)^2 = 0
    \right]
    \, ,
$$
with $p > N_n$.

Now we have that $|\mathcal{N}_i| \leq 2^{b_i} = \max(s_i)$ .
Meaning $s_i = \sum_{k=0}^{b_i} 2^k q_k$ with $ q_k = 0, 1$.

\begin{align}
    \sum_{i=0}^{N_n-1} \left(x_i - s_{i} - 1 + \sum_{j \in\mathcal{N}_i} x_j\right)^2
    &=
    \alpha_{i_1 i_2} x_{i_1} x_{i_2}
    + \beta_{i_1 i_2} x_{i_1} q_{k_2}
    + \gamma_{i_1 i_2} q_{k_1} q_{k_2}
    + 1
    =
    \psi Q \psi + 1
\end{align}
such that
$$
    Q = \begin{pmatrix}\alpha & \beta \\ 0 & \gamma \end{pmatrix}
    \, , \qquad
    \psi = \begin{pmatrix} \vec x \\ \vec q\end{pmatrix}
$$

With

\begin{align}
    \alpha \leftrightarrow & 
        x_i^2 + 2 x_i \sum_j x_j  - 2 x_i - 2 \sum_j x_j + \left(\sum_j x_j\right)^2
    \, , &
    \beta \leftrightarrow & 
        - 2 x_i s_i - 2 s_i \sum_j x_j
    \, , &
    \gamma \leftrightarrow & \,
       s_i^2 + 2 s_i
    \\
    \alpha = & \mathbb{1} + 2 N - 2 \mathbb{1} - 2 \text{diag}(|N|) + N^2
    \, , &
    \beta = & -2 (\mathbb{1} + N) B
    \, , &
     \gamma = & \, B^T\mathbb{1}B + 2 \text{diag}(|B|)
\end{align}

where $N_{ij}$ is the adjecency matrix of the graph (symmetric and zero diagonal) and $B_{ik}$ is the bit map for
integers $s_i$  to bits $q_k$ (note that the map encodes different $s_i$ with different length bits).

## Details

\begin{align}
    \sum_{i \in V} x_i^2 &\to \delta_{ij} \,,&
    \sum_{i \in V} x_i & \to \delta_{ij} \,,
\end{align}

\begin{align}
    \sum_{i \in V} \sum_{j \in \mathcal N_i} x_j & \to \sum_{k} N_{kj} \delta_{ij} \, , &
    \sum_{i \in V} \sum_{j \in \mathcal N_i} x_i x_j & \to N_{ij} \, , &
    \sum_{i_1 \in V}\sum_{j_1 \in\mathcal{N}_{i_1}} x_{j_1} \sum_{j_2 \in\mathcal{N}_{i_1}} x_{j_2}
    & \to
   N_{i j_1} N_{i j_2}
   =
   N_{j_1 i} N_{i j_2} \, ,
\end{align}

\begin{align}
   \sum_{i \in V} \sum_{j \in \mathcal N_i} s_i x_j & \to \sum_k N_{ji} B_{ik}  \, , &
\end{align}
where $N_{ij}$ is the adjecency matrix of the graph (symmetric and zero diagonal).

$$
     \sum_{j \in \mathcal N_i} x_j = \vec n_i \cdot \vec x  = \sum_{j \in V} N_{ij} x_j\, ,
$$

where $\vec n_i$ is a vector of ones and zeros---being one if vertx $j$ is a neighbor of vertex $i$.
I believe $N$ is also called the adjancency matrix of the graph.

$$
    N_{ij} = \begin{cases}
        1 & j \in \mathcal N_i \\ 0 & \text{otherwise}
    \end{cases}
$$

Note that $i$ is not in its own neighborhood $\mathcal N_i$.

Thus
\begin{align}
    \sum_{i \in V} x_i c_i
    &=
    \vec c \cdot \vec x
    \, , &
    c_i = \vec n_i \cdot \vec x
    & & \Rightarrow&  &
    \sum_{i \in V} \sum_{j \in \mathcal N_i} x_i x_j
    &=
    \vec x \cdot N \cdot \vec x 
\end{align}

and accordingly

\begin{align}
    \sum_{i \in V} \sum_{j \in\mathcal{N}_{i}} x_{j}
    & \to
    \sum_{j \in V}|\mathcal N_j| x_j
\end{align}
where $|\mathcal N_j|$ is the number of neighbors of vertex $j$.

## Example

Picture the graph
$$
 0 - 1 - 2
$$

Thus we have
\begin{align}
    N &= \begin{pmatrix}
        0 & 1 & 0 \\ 1 & 0 & 1 \\ 0 & 1 & 0
    \end{pmatrix}
   \, , &
   |N| = \begin{pmatrix}
        1 \\ 2 \\ 1
   \end{pmatrix}
   \, .
\end{align}

The slack variables must be $\vec s \geq |N|$. Thus the bit map is given by
\begin{align}
    s_1 &= 2^0 q_{1 0} \Rightarrow \max(s_1) \geq 1 \, , & 
    s_2 &= 2^0 q_{2 0} + 2^1 q_{2 1} \Rightarrow \max(s_2) \geq 2 \, , & 
    s_3 &= 2^0 q_{3 0} \Rightarrow \max(s_3) \geq 1 \, , & 
\end{align}
with $q_{ik} \in \{0,1\}$.
We therefor get
\begin{align}
    B &= \begin{pmatrix}
        1 & 0 & 0 & 0 \\ 0 & 1 & 2 & 0 \\ 0 & 0 & 0 & 1
    \end{pmatrix}
   \, , &
   \vec q
   &=
   \begin{pmatrix}
        q_{1 0} \\ q_{2 0} \\ q_{2 1} \\ q_{3 0}
   \end{pmatrix}
   \, , &
   |B| = \begin{pmatrix}
        1 \\ 1 \\ 2 \\ 1
   \end{pmatrix}
   \, .
\end{align}



In [ ]:
graph = {(0, 1), (1, 2)}

adjacency = dok_matrix((3, 3), dtype=int)

for edge in graph:
    adjacency[edge] = 1
adjacency += adjacency.T
print("N = \n", adjacency.todense())
print("\n|N| = \n", adjacency.sum(axis=1))

bitmap = dok_matrix((3, 4), dtype=int)
bitmap[(0, 0)] = 1
bitmap[(1, 1)] = 1
bitmap[(1, 2)] = 2
bitmap[(2, 3)] = 1


print("B = \n", bitmap.todense())

In [ ]:
one = dok_matrix(adjacency.shape, dtype=int)
for n in range(adjacency.shape[0]):
    one[(n, n)] = 1

neighborhood = dok_matrix(adjacency.shape, dtype=int)
for n, el in enumerate(adjacency.sum(axis=1)):
    neighborhood[(n, n)] = el

alpha = -one + 2 * adjacency - 2 * neighborhood + adjacency @ adjacency

In [ ]:
from sympy import S, Matrix

In [ ]:
x1, x2, x3 = S("x1"), S("x2"), S("x3")
M = Matrix(alpha.todense())
x = Matrix([[x1, x2, x3]]).T

print(x)

sol = (x.T @ M @ x).expand()[0] + 3
sol

In [ ]:
nn = Matrix([x2, x1 + x3, x2])

constraint = Matrix([x1 - 1 + x2, x2 - 1 + x1 + x3, x3 - 1 + x2])
eq = (constraint.T @ constraint).expand()[0]
eq

In [ ]:
eq - sol

In [ ]:
q10, q20, q21, q30 = S("q10"), S("q20"), S("q21"), S("q30")
q = Matrix([[q10, q20, q21, q30]]).T

beta = -2 * (one + adjacency) @ bitmap
beta.todense()

In [ ]:
s = Matrix([["s1", "s2", "s3"]]).T
sol = (x.T @ beta @ q)[0].subs({"q10": "s1", "q30": "s3", "q20": "s2 - 2*q21"}).expand()
sol

In [ ]:
eq = (-2 * x.T @ s - 2 * nn.T @ s)[0].expand()
eq

In [ ]:
eq - sol

In [ ]:
bb = dok_matrix((4, 4), dtype=int)
for n, el in enumerate(bitmap.sum(axis=0).tolist()[0]):
    bb[(n, n)] = el

gamma = bitmap.T @ bitmap + 2 * bb
sol = (q.T @ gamma @ q)[0].expand()
sol
gamma

In [ ]:
sol - eq

In [ ]:
print(alpha.todense())
print(beta.todense())
print(gamma.todense())

## QUBO

In [ ]:
def get_adjacency(graph: Set[Tuple[int]]) -> dok_matrix:
    """This routine computes the adjecency matrix for a given graph.
    
    It assumes the graph is connected and nodes are labeled from 0...N-1.
    Self loops are forbidden.
    """
    if not isinstance(graph, set):
        raise TypeError("Graph must be a set of tuples")

    nodes = set()
    adj = dict()

    for v1, v2 in graph:
        if v1 == v2:
            raise KeyError(f"Self loops are not allowed (v={v1}).")

        nodes.add(v1)
        nodes.add(v2)
        adj[(v1, v2)] = 1

    nodes = sorted(nodes)
    n_nodes = len(nodes)

    if nodes != list(range(n_nodes)):
        raise KeyError("Nodes must be from 0 ... n_nodes")

    adjacency = dok_matrix((n_nodes, n_nodes), dtype=int)
    for (v1, v2), val in adj.items():
        adjacency[(v1, v2)] = val
        adjacency[(v2, v1)] = val

    if not all([val == 1 for val in adjacency.values()]):
        raise KeyError("Double entries in graph")

    return adjacency


adjacency = get_adjacency(test_graph)
adjacency.todense()

In [ ]:
adjacency = get_adjacency(test_graph)
n_neigbors = adjacency.sum(axis=1).flatten().tolist()[0]
print(n_neigbors)


def get_bitmap(n_neigbors: List[int]) -> dok_matrix:
    r"""Computes the map of slack-bits to slack integers.
    
    This matrix B maps the bit vector b to slack variables s.
    Each s_i has max(s_i) >= N_i where N_i is the number of 
    neigbors of vertex i.  
    The bit vector b = (b_00, b_01, b_02, ... , b_11, b_12, ...) 
    with b_ij will be mapped onto s_iaccording to 
    s_i = sum_j 2**j b_ij.
    Thus the size of the returned matrix scales as 
    ~ sum_i log2(N_i)
    
    Arguments:
        n_neigbors: The number of neighbors for each vertex.
    """
    n_nodes = len(n_neigbors)

    # Figure out how many bits we need for each neighborhood
    n_bits = np.floor(np.log2(n_neigbors) + 1).astype(int)
    
    print(n_bits)

    bitmap = dok_matrix((n_nodes, n_bits.sum()), dtype=int)

    acc = 0
    for v, n_bit in enumerate(n_bits):
        for n in range(n_bit):
            bitmap[(v, acc)] = 2 ** n
            acc += 1

    return bitmap


bitmap = get_bitmap(n_neigbors)
bitmap.todense()

In [ ]:
def get_mds_qubo(graph: Set[Tuple[int]]) -> dok_matrix:
    """This routine computes minimal dominating set QUBO for a given graph.
    
    It assumes the graph is connected and nodes are labeled from 0...N-1.
    Self loops are forbidden.
    """
    ## This is N
    adjacency = get_adjacency(graph)
    
    ## Id in x-space
    one = dok_matrix(adjacency.shape, dtype=int)
    for n in range(adjacency.shape[0]):
        one[(n, n)] = 1
       
    ## |N|
    n_neigbors = adjacency.sum(axis=1).flatten().tolist()[0]
    
    ## diag(|N|)
    diag_neighbor = dok_matrix(adjacency.shape, dtype=int)
    for n, el in enumerate(n_neigbors):
        diag_neighbor[(n, n)] = el
        
    ## B
    bitmap = get_bitmap(n_neigbors)
    n_bits = bitmap.shape[1]
        
    ## diag(|B|)
    diag_bitmap = dok_matrix((n_bits, n_bits), dtype=int)
    for n, el in enumerate(bitmap.sum(axis=0).tolist()[0]):
        diag_bitmap[(n, n)] = el
        
    ## Compute QUBO components
    alpha = -one + 2 * adjacency - 2 * diag_neighbor + adjacency @ adjacency
    beta = -2 * (one + adjacency) @ bitmap
    gamma = bitmap.T @ bitmap + 2 * diag_bitmap
    
    ## Multiply by penalty factor
    penalty = len(n_neigbors) + 1
    alpha *= penalty
    beta *= penalty
    gamma *= penalty
    
    ## Add in minimization condition
    alpha += one
    
    ## Construt QUBO
    return bmat([[alpha, beta], [None, gamma]]).todok()
    
    
qubo = get_mds_qubo(test_graph)
qubo

In [ ]:
energies = []
vecs = []
for vv in product(*[(0, 1)] * qubo.shape[1]):
    ee = (vv @ qubo @ vv).flatten()[0]
    energies.append(ee)
    vecs.append(vv)

e_min = np.min(energies)
solutions = [
    [nn for nn, bb in enumerate(vv[:n_nodes]) if bb == 1]
    for vv, ee in zip(vecs, energies)
    if ee == e_min
]
solutions

In [ ]:
for sol in solutions:
    plot = get_plot(test_graph, color_nodes=sol)
    show(plot)